In [1]:
#Slef-made Function
from Dataload import dataload
#from module import transformer,video_model

from module import video_model_attention
from constant import EMOTIPATH,EMOTIFACEPATH,EMOTIAUDIOPATH
from train import methods


#Torch Library

import torch
import torch.nn as nn
from torchsummary import summary
from torch.utils.data import Dataset, DataLoader
from torchvision import  utils

#Sub tools
import pandas as pd

import os
import matplotlib.pylab as plt

#Util Library
import time
from tqdm.notebook import tqdm
from mpl_toolkits.axes_grid1 import ImageGrid  

Train_label=os.path.join(EMOTIPATH,"Train_labels.txt")
Train_video_pt=os.path.join(EMOTIPATH,"pt","Train")
Train_face_pt=os.path.join(EMOTIFACEPATH,"pt_stacked","Train")
Audio_Train=os.path.join(EMOTIAUDIOPATH,"training_audio_compare_filtered.csv")

Val_labels=os.path.join(EMOTIPATH,"Val_labels.txt")
Val_video_pt=os.path.join(EMOTIPATH,"pt","Valid")
Val_face_pt=os.path.join(EMOTIFACEPATH,"pt_stacked","Valid")
Audio_Valid=os.path.join(EMOTIAUDIOPATH,"validation_audio_compare_filtered.csv")


train_table = pd.read_csv(Train_label,delimiter=' ')
val_table = pd.read_csv(Val_labels,delimiter=' ')
print(train_table['Label'].value_counts())
print(val_table['Label'].value_counts())



device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.set_default_tensor_type(torch.cuda.FloatTensor)



frame_num=10

train_data_pt=dataload.Video_Frame_Data(Train_label,base_path_v=Train_video_pt,face_path=Train_face_pt,frame_num=frame_num,direct=True,audio_csv=Audio_Train)
valid_data_pt=dataload.Video_Frame_Data(Val_labels,base_path_v=Val_video_pt,face_path=Val_face_pt,frame_num=frame_num,direct=True,audio_csv=Audio_Valid)





def load_pretrained_model(pre_train):
    face_model=torch.load("pre_trained_model/" +"pre_trained_resnet18_face.h")
    
   # face_model=torch.load("pre_trained_model/" +"pre_trained_resnet18_facev2.h")
    audio_model=video_model_attention.AudioRecognition(softmax=pre_train)
   # audio_model.load_state_dict(torch.load("pre_trained_model/pre_embedded_audio.pth"),strict=False)
    image_model=video_model_attention.Video_modeller(frame_num,face_model=face_model,pre_train=pre_train)
   # image_model.load_state_dict(torch.load("saved_model_img/img_global_5.pth"),strict=False)
    return(image_model,audio_model)



def show_img(dataset,index,frame):
    x=dataset[index][0][frame].cpu().numpy()
    plt.figure(1,(15,15))
    plt.axis('off')
    image = (x*0.5+0.5).transpose((1, 2, 0)).squeeze()
    plt.imshow(image)
    
    fig = plt.figure(2,(15,15))
   # plt.axis('off')
    grid = ImageGrid(fig, 111,
                     nrows_ncols=(1,5),
                     axes_pad=0.1,
                     )
    for i in range(5):
        face=dataset[index][1][frame].cpu().numpy()
        image = (face[i,:]*0.5+0.5).transpose((1, 2, 0)).squeeze()
        grid[i].imshow(image,cmap='gray',interpolation='none')
        
#img_model,audi=load_pretrained_model(True)
audio_model=video_model_attention.AudioRecognition(softmax=True)


3    932
2    923
1    806
Name: Label, dtype: int64
1    299
2    281
3    186
Name: Label, dtype: int64


In [2]:
df1=pd.read_csv(os.path.join(EMOTIAUDIOPATH,"training_audio_compare.csv"))

In [3]:
df1

,Unnamed: 0,file_name,audspec_lengthL1norm_sma_maxPos,audspec_lengthL1norm_sma_minPos,audspec_lengthL1norm_sma_quartile1,audspec_lengthL1norm_sma_quartile2,audspec_lengthL1norm_sma_quartile3,audspec_lengthL1norm_sma_iqr1-2,audspec_lengthL1norm_sma_iqr2-3,audspec_lengthL1norm_sma_iqr1-3,...,mfcc_sma_de[14]_peakRangeAbs,mfcc_sma_de[14]_peakRangeRel,mfcc_sma_de[14]_peakMeanAbs,mfcc_sma_de[14]_peakMeanMeanDist,mfcc_sma_de[14]_peakMeanRel,mfcc_sma_de[14]_minRangeRel,mfcc_sma_de[14]_meanRisingSlope,mfcc_sma_de[14]_stddevRisingSlope,mfcc_sma_de[14]_meanFallingSlope,mfcc_sma_de[14]_stddevFallingSlope
0,0,101_1.arff,0.820565,0.074597,1.533161,2.116676,2.989286,0.583515,0.872610,1.456125,...,11.437800,0.501672,2.920215,2.910282,294.00100,0.570569,112.7632,57.08240,128.33450,90.23294
1,0,101_10.arff,0.860887,0.000000,1.541169,2.171726,2.637154,0.630556,0.465429,1.095985,...,8.345563,0.612859,2.596471,2.602683,-418.00840,0.499175,126.8875,61.04508,123.18180,51.11492
2,0,101_11.arff,0.133065,0.820565,0.749147,1.447200,2.007862,0.698052,0.560662,1.258715,...,7.392002,0.444427,2.511684,2.542728,-80.90839,0.582766,124.6919,57.76358,123.43000,62.07063
3,0,101_12.arff,0.973790,0.260081,1.201816,1.745465,2.246745,0.543649,0.501280,1.044929,...,6.569743,0.450158,2.708047,2.713984,-456.18710,0.596681,119.6158,67.20280,118.46220,60.10581
4,0,101_13.arff,0.137097,0.877016,2.149037,2.760834,3.297577,0.611797,0.536744,1.148541,...,7.430662,0.501442,2.882833,2.897373,-198.26420,0.549199,125.5966,62.50634,134.31670,61.63183
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2656,0,9_5.arff,0.520161,0.381048,0.329917,0.544102,0.764169,0.214186,0.220067,0.434253,...,5.151070,0.409382,2.392092,2.396713,-517.61520,0.687082,119.5502,59.00103,115.59010,62.14444
2657,0,9_6.arff,0.766129,0.614919,0.171487,0.404498,0.947121,0.233012,0.542622,0.775634,...,6.132486,0.531017,2.072510,2.096777,-85.40347,0.501981,106.8260,47.53175,103.10680,44.40643
2658,0,9_7.arff,0.451613,0.921371,0.001963,0.155606,0.561355,0.153643,0.405749,0.559392,...,5.070104,0.518415,1.717270,1.714476,614.52500,0.477875,88.2767,43.20261,89.59177,48.38278
2659,0,9_8.arff,0.616935,0.147177,0.793189,1.099609,1.490128,0.306420,0.390518,0.696939,...,5.669058,0.460134,2.504291,2.507518,-775.96900,0.595428,116.2459,55.57997,105.34260,40.21917


In [4]:
list_na=df1.columns[df1.isna().any()].tolist()

In [5]:
list_na

['pcm_fftMag_spectralSlope_sma_de_lpgain',
 'pcm_fftMag_spectralSlope_sma_de_lpc0',
 'pcm_fftMag_spectralSlope_sma_de_lpc1',
 'pcm_fftMag_spectralSlope_sma_de_lpc2',
 'pcm_fftMag_spectralSlope_sma_de_lpc3',
 'pcm_fftMag_spectralSlope_sma_de_lpc4']

In [6]:


weights=[932/932,932/923,932/806]
class_weights = torch.FloatTensor(weights).to(device)

train_dataloader = DataLoader(train_data_pt, batch_size=32
                       , num_workers=0,shuffle=True)
valid_dataloader = DataLoader(valid_data_pt, batch_size=32
                   , num_workers=0)
#model=video_model.video_transformer(img_model,audio_model)
model=audio_model
model=model.to(device)

num_epochs=50
criterion = nn.CrossEntropyLoss(weight=class_weights)  

for name, child in model.named_children():
    if not name in ['face_model','frame_model' ]:
        print(name + ' is unfrozen')
        for param in child.parameters():
            param.requires_grad = True
    else:
        print(name + ' is frozen')
        for param in child.parameters():
               param.requires_grad = False

fc1 is unfrozen
fc2 is unfrozen
fc3 is unfrozen
relu is unfrozen
tanh is unfrozen


In [7]:
model(train_data_pt[8][2].unsqueeze(0).float().to(device))

tensor([[0., 1., 0.]], grad_fn=<SoftmaxBackward>)

In [8]:
train_data_pt[6][2]

tensor([ 0.5242,  0.5887,  0.5940,  ..., 50.9046, 95.2676, 45.5413],
       device='cpu', dtype=torch.float64)

In [9]:

optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.0005, betas=(0.5, 0.999))


model_name="audio_train_v2"

print(model_name)

methods.audio_train("audio_train_v2",num_epochs,model_name,model,train_dataloader,valid_dataloader,optimizer,criterion,device)


audio_train_v2
Epoch:  1 

Validation 

Validation Loss:  1.1905978148182232
Validation Accuracy:  0.3668407310704961
Confusion Matrix: 
 [[  0 299   0]
 [  0 281   0]
 [  0 186   0]]


Batch:  40 / 84
Batch Recognition loss:  1.2826348543167114
Batch:  80 / 84
Batch Recognition loss:  1.3451135158538818

Average_Loss:  1.2213762516067141
Average_Accuracy:  0.34723788049605414
Confusion Matrix: 
 [[  2 803   1]
 [  0 922   1]
 [  0 932   0]]
Epoch:  2 

Validation 

Validation Loss:  1.1911175573865573
Validation Accuracy:  0.3668407310704961
Confusion Matrix: 
 [[  0 299   0]
 [  0 281   0]
 [  0 186   0]]


Batch:  40 / 84
Batch Recognition loss:  1.2838720083236694
Batch:  80 / 84
Batch Recognition loss:  1.3753430843353271

Average_Loss:  1.2189583381017048
Average_Accuracy:  0.34723788049605414
Confusion Matrix: 
 [[  2 803   1]
 [  0 922   1]
 [  0 932   0]]
Epoch:  3 

Validation 

Validation Loss:  1.1911175573865573
Validation Accuracy:  0.3668407310704961
Confusion Matrix: 
 [[  0 299   0]
 [  0 281   0]
 [  0 186   0]]


Batch:  40 / 84
Batch Recognition loss:  1.3712489604949951
Batch:  80 / 84
Batch Recognition loss:  1.3469796180725098

Average_Loss:  1.2192763899053847
Average_Accuracy:  0.34723788049605414
Confusion Matrix: 
 [[  2 803   1]
 [  0 922   1]
 [  0 932   0]]
Epoch:  4 

Validation 

Validation Loss:  1.1911175573865573
Validation Accuracy:  0.3668407310704961
Confusion Matrix: 
 [[  0 299   0]
 [  0 281   0]
 [  0 186   0]]


Batch:  40 / 84
Batch Recognition loss:  1.167160987854004
Batch:  80 / 84
Batch Recognition loss:  1.1653939485549927

Average_Loss:  1.2210407767977034
Average_Accuracy:  0.34723788049605414
Confusion Matrix: 
 [[  2 803   1]
 [  0 922   1]
 [  0 932   0]]
Epoch:  5 

Validation 

Validation Loss:  1.1911175573865573
Validation Accuracy:  0.3668407310704961
Confusion Matrix: 
 [[  0 299   0]
 [  0 281   0]
 [  0 186   0]]


...

Batch:  40 / 84
Batch Recognition loss:  1.1319609880447388
Batch:  80 / 84
Batch Recognition loss:  1.1966198682785034

Average_Loss:  1.2154504507780075
Average_Accuracy:  0.34723788049605414
Confusion Matrix: 
 [[  2 803   1]
 [  0 922   1]
 [  0 932   0]]
Epoch:  11 

Validation 

Validation Loss:  1.1911175573865573
Validation Accuracy:  0.3668407310704961
Confusion Matrix: 
 [[  0 299   0]
 [  0 281   0]
 [  0 186   0]]


Batch:  40 / 84
Batch Recognition loss:  1.3469798564910889
Batch:  80 / 84
Batch Recognition loss:  1.1599953174591064

Average_Loss:  1.2191288598946162
Average_Accuracy:  0.34723788049605414
Confusion Matrix: 
 [[  2 803   1]
 [  0 922   1]
 [  0 932   0]]
Epoch:  12 

Validation 

Validation Loss:  1.1911175573865573
Validation Accuracy:  0.3668407310704961
Confusion Matrix: 
 [[  0 299   0]
 [  0 281   0]
 [  0 186   0]]


Batch:  40 / 84
Batch Recognition loss:  1.194988489151001
Batch:  80 / 84
Batch Recognition loss:  1.2595986127853394

Average_Loss:  1.219390652009419
Average_Accuracy:  0.34723788049605414
Confusion Matrix: 
 [[  2 803   1]
 [  0 922   1]
 [  0 932   0]]
Epoch:  13 

Validation 

Validation Loss:  1.1911175573865573
Validation Accuracy:  0.3668407310704961
Confusion Matrix: 
 [[  0 299   0]
 [  0 281   0]
 [  0 186   0]]


Batch:  40 / 84
Batch Recognition loss:  1.1933413743972778
Batch:  80 / 84
Batch Recognition loss:  1.3432132005691528

Average_Loss:  1.2231564621130626
Average_Accuracy:  0.34723788049605414
Confusion Matrix: 
 [[  2 803   1]
 [  0 922   1]
 [  0 932   0]]
Epoch:  14 

Validation 

Validation Loss:  1.1911175573865573
Validation Accuracy:  0.3668407310704961
Confusion Matrix: 
 [[  0 299   0]
 [  0 281   0]
 [  0 186   0]]


RuntimeError: CUDA out of memory. Tried to allocate 150.00 MiB (GPU 0; 10.75 GiB total capacity; 6.07 GiB already allocated; 129.00 MiB free; 117.38 MiB cached)